In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.linear_model import LinearRegression

In [9]:
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, ReLU, ELU, PReLU, Input, Concatenate, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import mse, binary_crossentropy
import tensorflow as tf

In [10]:
def myloss2d(y_true, y_pred, alpha = 0.0):
    #alpha determines the amount of decorrelation; 0 means no decorrelation.
    
    #We want to learn f(g(x)) = x with g != identity and g(x) and x should have the same probability density.
    #g(x) = y_pred[:,0]
    #f(g(x)) = y_pred[:,1]
    #h(x) = y_pred[:,2]
    #h(g(x)) = y_pred[:,3]
    
    myoutput =  mse(y_true[:,0],y_pred[:,1]) \
                - alpha*binary_crossentropy(y_pred[:,2],K.ones_like(y_pred[:,2])) \
                - alpha*binary_crossentropy(y_pred[:,3],0.*K.ones_like(y_pred[:,2]))
    return myoutput

In [11]:
num_iters = 10
training = 20
slopes = []
intercepts = []

for j in range(num_iters):
    print("on j=",j)
    x0 = np.random.normal(0,1,100000)
    x1 = np.random.normal(0,2,100000)
    X = np.c_[x0, x1]
    np.random.shuffle(X)
    
    myinput_2d = Input(shape=(2,))
    encoded_2d = Dense(5, activation='elu')(myinput_2d)
    encoded_2d = Dense(5, activation='elu')(encoded_2d)
    encoded_2d = Dense(2, activation='linear')(encoded_2d)

    encoder_2d = Model(myinput_2d, encoded_2d)
    encoder2_2d = encoder_2d(encoder_2d(myinput_2d))
    autoencoder_2d = Model(myinput_2d, encoder2_2d)

    combinedModel_2d = Model(myinput_2d,Concatenate(axis=-1)([encoded_2d, encoder2_2d]))

    myinput_classify_2d = Input(shape=(2,))
    myclassifier_2d = Dense(128, activation='elu')(myinput_classify_2d)
    myclassifier_2d = Dense(64, activation='elu')(myclassifier_2d)
    myclassifier_2d = Dense(1, activation='sigmoid')(myclassifier_2d)
    myclassifier_model_2d = Model(myinput_classify_2d, myclassifier_2d)
    myclassifier_input_2d = myclassifier_model_2d(myinput_2d)
    myclassifier_encoded_2d = myclassifier_model_2d(encoded_2d)

    combinedModel_classifier_2d = Model(myinput_2d,Concatenate(axis=-1)([encoded_2d, encoder2_2d, myclassifier_input_2d, myclassifier_encoded_2d]))

    preds = []
    preds += [encoder_2d.predict(X)]

    for i in range(training):
        #Now, train the classifier
        encoded_x_2d = encoder_2d.predict(X)
        myclassifier_model_2d.compile(optimizer='adam', loss='binary_crossentropy')
        myclassifier_model_2d.fit(np.concatenate([X,encoded_x_2d]),np.concatenate([np.ones(len(X)),np.zeros(len(X))]), epochs=1, batch_size=int(0.01*len(X)), verbose=0)

        #Now, update the autoencoder
        for layer in myclassifier_model_2d.layers[:]:
            layer.trainable = False

        combinedModel_classifier_2d.compile(optimizer='adam', loss=lambda y_true, y_pred: myloss2d(y_true, y_pred))
        combinedModel_classifier_2d.fit(X,X, epochs=1, batch_size=int(0.01*len(X)), verbose=0)

        autoencoder_2d.compile(optimizer='adam', loss='mse')
        autoencoder_2d.fit(X,X, epochs=5, batch_size=int(0.01*len(x0)), verbose=0)

        preds += [encoder_2d.predict(X)]

        for layer in myclassifier_model_2d.layers[:]:
            layer.trainable = True
            
    model = LinearRegression().fit(X, preds[-1]);
    slopes.append(model.coef_);
    intercepts.append(model.intercept_)

on j= 0
on j= 1
on j= 2
on j= 3
on j= 4
on j= 5
on j= 6
on j= 7
on j= 8
on j= 9


In [12]:
for i in range(num_iters):
    det = np.linalg.det(slopes[i])
    print(round(det, 0))

-1.0
-1.0
1.0
-1.0
-1.0
1.0
-1.0
1.0
-1.0
-1.0


In [14]:
print("Slopes:")
for i in range(num_iters):
    s = [[str(e) for e in row] for row in slopes[i]]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print ("\n".join(table))
    print("\n")
    
print("Intercepts:")
for i in range(num_iters):
    s = [[str(e) for e in row] for row in intercepts[i]]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print ("\n".join(table))
    print("\n")
    

Slopes:
0.06962446050367109	0.933087759689277   
1.0777495264494905 	-0.06570244631557372


0.18956008990592246	0.7991549402713046
1.176512627479619  	-0.234879688289236


0.9562315967874372   	0.0019884422446651852
-0.006066091471150463	0.997046273725789    


0.07641014838057246	0.899795591846032  
1.1041193760120864 	-0.0543557408035767


0.1498248873993785	0.8802004769845915  
1.083937105504562 	-0.15034884648569163


-1.0228627144685734 	0.041972934068507724
0.045233670427428437	-1.0264715231944748 


0.01882838028939675	0.9295090353077243  
1.0920662123145493 	-0.02111380535408919


-0.9842594803648426	-0.00447898192075484
0.06888135448450132	-0.9053270552874787 


0.20905193631180471	0.8825384393810982  
1.1088025311063543 	-0.20586950329053302


0.008974249495298592	0.9968630407454412   
1.0163627498482484  	-0.027057895248937803


Intercepts:


TypeError: 'numpy.float64' object is not iterable